<h2>Centerline</h2>

In this project, we use a centerline during the training phase to help the agent to avoid walls and to follow the track.

One way of defining such a trajectory is to make a human play the track safetly and save car's data at regular timesteps.

We also use states of the game to rewind the car at a random positions on the track.

In [15]:
import os
import time
import pickle
import keyboard
import numpy as np
import random

import tminterface as tmi
from tminterface.interface import TMInterface, Client

In [16]:
# Instantiate the Interface
# The client is used to connect the interface to the TMI server
# The interface runs itself in a separate thread

interface = TMInterface()
client = Client()

interface.register(client)

True

In [17]:
state = interface.get_simulation_state()

In [5]:
# HELPERS 

# Dynamic state of the car. Position, rotation, speed, etc...
#print(state.dyna.current_state)

# Data about wheels (index 1 out of 4). Contact with the ground, sliding...
#print(state.simulation_wheels[0])

# Time in milisseconds
#print(state.time / 1000 / 60)

In [264]:
TRACK_DATA_PATH = "track_data/"
track_name = "Training_dataset_tech"

def create_folder_path(track_name):
    
    track_folder = os.path.join(TRACK_DATA_PATH, track_name)
    
    if os.path.exists(track_folder):
        n_run = len(os.listdir(track_folder))
    else:
        os.makedirs(track_folder)
        n_run = 0
    
    run_folder = os.path.join(track_folder, 'run-%s' % (n_run + 1))
    
    os.makedirs(run_folder)
    
    return run_folder

## Main Loop used to capture game data

In [265]:
run_folder = create_folder_path(track_name=track_name)

# Time in milisseconds
save_position_every = 100
save_state_every = 5_000

state = interface.get_simulation_state()
# Wait for the user to move to start saving data
print("Waiting for player to move")
while state.race_time >= 0:
    state = interface.get_simulation_state()

interface.execute_command("warp 0")
state = interface.get_simulation_state()

# Setup initial position and state just after the player starts to move
positions = [{
    'time': None,
    'position': state.dyna.current_state.position
}]
pickle.dump(state, open(os.path.join(run_folder, f"state_{0:05d}.pkl"), "wb"))
n_saved_states = 1

positions[0]['time'] = state.time

print("Start to record data")

start_time = state.time

while True:
    
    state = interface.get_simulation_state()
    
    if (state.time - start_time) / save_position_every > len(positions):
        positions.append({
            'time': state.time,
            'position': state.dyna.current_state.position
        })
        
    if (state.time - start_time) / save_state_every > n_saved_states:
        pickle.dump(state, open(os.path.join(run_folder, f"state_{n_saved_states:05d}.pkl"), "wb"))
        n_saved_states += 1

    time.sleep(.1)

    if keyboard.is_pressed("q"):
        print("Stop recording")
        break
        
pickle.dump(positions, open(os.path.join(run_folder, "positions.pkl"), "wb"))

Waiting for player to move
Start to record data
Stop recording


## Rewind the game to a specific state

In [29]:
run_folder = "track_data/Training_dataset_tech/run-2"
state_files = list(filter(lambda x: x.startswith("state"), os.listdir(run_folder)))
# random.shuffle(state_files)

for state_file in state_files :
    state = pickle.load(open(os.path.join(run_folder, state_file), "rb"))
    state.dyna.current_state.linear_speed = np.array([0, 0, 0])
    interface.rewind_to_state(state)
    time.sleep(1)

# for k in range(3):
#     state_file = state_files[8]
#     print(state_file)
#     state = pickle.load(open(os.path.join(run_folder, state_file), "rb"))
#     state.dyna.current_state.linear_speed = np.array([0, 0, 0])
#     interface.rewind_to_state(state)
#     time.sleep(1)

## Load positions

In [2]:
run_folder = "track_data/Training_dataset_tech/run-2"
positions = pickle.load(open(os.path.join(run_folder, "positions.pkl"), "rb"))
print(positions)

[{'time': 0, 'position': [ArrayFieldProxy object at 0x13ce690c8f0]
Data: [1012.7999877929688 10.210000038146973 624.0]}, {'time': 2600, 'position': [ArrayFieldProxy object at 0x13ce61da480]
Data: [1012.7999877929688 10.210000038146973 624.0]}, {'time': 2720, 'position': [ArrayFieldProxy object at 0x13ce61d9e50]
Data: [1012.7999877929688 10.061371803283691 624.0]}, {'time': 2820, 'position': [ArrayFieldProxy object at 0x13ce61da0f0]
Data: [1012.8001098632812 10.003349304199219 624.0004272460938]}, {'time': 2930, 'position': [ArrayFieldProxy object at 0x13ce6963b30]
Data: [1012.8001708984375 10.005057334899902 624.0010986328125]}, {'time': 3030, 'position': [ArrayFieldProxy object at 0x13ce6963d10]
Data: [1012.7271118164062 10.007074356079102 624.0011596679688]}, {'time': 3140, 'position': [ArrayFieldProxy object at 0x13ce6962ba0]
Data: [1012.460205078125 10.013745307922363 624.0008544921875]}, {'time': 3240, 'position': [ArrayFieldProxy object at 0x13ce69743b0]
Data: [1012.0537719726562

In [24]:
list(state.dyna.current_state.position.to_numpy())

[1012.8001708984375, 10.013981819152832, 624.001220703125]